In [131]:
import configparser 
import logging
import datetime as dt
from influxdb import InfluxDBClient, DataFrameClient
import json

In [9]:
config = configparser.ConfigParser()
config.read('./configs/monitor.conf')

['./configs/monitor.conf']

In [13]:
def get_influxdb_clients():
    # FIXME
    influxdb_client = InfluxDBClient(
        config.get('INFLUXDB', 'ADDRESS'),
        config.get('INFLUXDB', 'PORT'),
        config.get('INFLUXDB', 'USER'),
        config.get('INFLUXDB', 'PASSWORD'),
        config.get('INFLUXDB', 'DB_NAME'))

    influxdb_dataframe_client = DataFrameClient(
        config.get('INFLUXDB', 'ADDRESS'),
        config.get('INFLUXDB', 'PORT'),
        config.get('INFLUXDB', 'USER'),
        config.get('INFLUXDB', 'PASSWORD'),
        config.get('INFLUXDB', 'DB_NAME'))

    return influxdb_client, influxdb_dataframe_client

In [237]:
uuid = '100316824873160941010780204477983883413'


def get_influxdb_data(client, df_client):

    last_element_query = 'select * from image order by desc limit 1'
    last_result = client.query(last_element_query)
    
    last_uuid = list(last_result.get_points())[0]['uuid']
    
    data_query = 'select * from image where uuid = \'{}\' group by "FILTER"  order by asc'.format(
        last_uuid)
    
#     data_query = 'select * from image group by "FILTER" order by asc limit 100'.format(
#                      uuid
#                     )
    result = df_client.query(data_query)
    result = dict(
        [(k[1][0][1], v.reset_index(level=0)) for k, v in result.items()])

    return last_result, result


In [238]:
client, df_client = get_influxdb_clients()

In [239]:
last_result, result = get_influxdb_data(client, df_client)

In [241]:
last_result

ResultSet({'('image', None)': [{'time': '2019-05-11T21:43:52.343364864Z', 'BACKGROUND': 1680.711, 'DATE-OBS': None, 'DEC_CENTER': 78.539425, 'DEC_HDR': 78.556122, 'EXPTIME': 10, 'EXPTIME_1': None, 'FILTER': 'B', 'FLUX_MAX': 7701.125, 'FWHM_IMAGE': 2.93, 'NUMBER': 35, 'OBJECT': 'excep', 'OBSERVER': 'MD', 'PHOTOMETRY_STATUS': 1, 'RA_CENTER': 30.653522, 'RA_HDR': 30.997612, 'SNR_WIN': 250.5, 'TIME-OBS': None, 'X_IMAGE': 294.1509, 'Y_IMAGE': 447.6156, 'image_time': '2019-03-19T20:33:14.09', 'uuid': '12815850787437882802703552300904228785'}]})

In [129]:
list(last_result.get_points())[0]

{'time': '2019-05-11T19:44:35.498585856Z',
 'BACKGROUND': 4892.284,
 'DATE-OBS': None,
 'DEC_CENTER': 36.936002,
 'DEC_HDR': 36.935278,
 'EXPTIME': 60,
 'EXPTIME_1': None,
 'FILTER': 'R',
 'FLUX_MAX': 2970.408,
 'FWHM_IMAGE': 2.18,
 'NUMBER': 130,
 'OBJECT': 'j0713',
 'OBSERVER': 'MD',
 'PHOTOMETRY_STATUS': 1,
 'RA_CENTER': 108.290449,
 'RA_HDR': 108.289583,
 'SNR_WIN': 88.67,
 'TIME-OBS': None,
 'X_IMAGE': 514.6434,
 'Y_IMAGE': 514.4751,
 'image_time': '2019-03-19T22:52:54.68',
 'uuid': '100316824873160941010780204477983883413'}

In [139]:
result['R']

,BACKGROUND,DEC_CENTER,DEC_HDR,EXPTIME,FLUX_MAX,FWHM_IMAGE,NUMBER,OBJECT,OBSERVER,PHOTOMETRY_STATUS,RA_CENTER,RA_HDR,SNR_WIN,X_IMAGE,Y_IMAGE,image_time,uuid
2019-05-11 19:43:32.586112+00:00,4804.625,36.935720,36.935278,60,2734.683,2.53,125,j0713,MD,1,108.290428,108.289583,80.42,514.7047,514.0812,2019-03-19T22:39:33.49,100316824873160941010780204477983883413
2019-05-11 19:43:38.244233984+00:00,4752.654,36.935945,36.935278,60,3034.291,2.31,129,j0713,MD,1,108.290489,108.289583,86.79,514.7081,514.2977,2019-03-19T22:40:35.09,100316824873160941010780204477983883413
2019-05-11 19:43:43.947181824+00:00,4762.810,36.935842,36.935278,60,3677.941,2.08,133,j0713,MD,1,108.290491,108.289583,88.18,514.7761,513.9423,2019-03-19T22:42:38.34,100316824873160941010780204477983883413
2019-05-11 19:43:49.757771008+00:00,4742.222,36.935947,36.935278,60,3091.632,2.15,134,j0713,MD,1,108.290423,108.289583,88.55,514.5234,514.2770,2019-03-19T22:43:40.05,100316824873160941010780204477983883413
2019-05-11 19:43:55.588613888+00:00,4720.176,36.935839,36.935278,60,3472.602,2.10,131,j0713,MD,1,108.290374,108.289583,90.49,514.4081,513.9408,2019-03-19T22:44:41.67,100316824873160941010780204477983883413
2019-05-11 19:44:01.238270976+00:00,4734.042,36.935905,36.935278,60,3082.198,2.29,131,j0713,MD,1,108.290460,108.289583,85.68,514.6035,514.1234,2019-03-19T22:45:43.28,100316824873160941010780204477983883413
2019-05-11 19:44:06.877198080+00:00,4734.333,36.935894,36.935278,60,3052.222,2.32,130,j0713,MD,1,108.290446,108.289583,84.52,514.6678,513.9885,2019-03-19T22:46:44.90,100316824873160941010780204477983883413
2019-05-11 19:44:12.534903040+00:00,4754.984,36.935899,36.935278,60,3257.117,2.26,129,j0713,MD,1,108.290461,108.289583,86.13,514.6646,514.1677,2019-03-19T22:47:46.52,100316824873160941010780204477983883413
2019-05-11 19:44:18.222999040+00:00,4764.438,36.935875,36.935278,60,3508.138,2.13,131,j0713,MD,1,108.290447,108.289583,89.51,514.6233,514.0405,2019-03-19T22:48:48.23,100316824873160941010780204477983883413
2019-05-11 19:44:23.813918208+00:00,4812.191,36.935870,36.935278,60,3199.462,2.28,131,j0713,MD,1,108.290424,108.289583,89.22,514.6265,514.0303,2019-03-19T22:50:51.45,100316824873160941010780204477983883413
